In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import datetime as dt
import re
import math
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### Thu thập dữ liệu

In [54]:
#REST API
'''
    Sử dụng api từ openweathermap.org
    Lấy dữ liệu chất lượng không khí tại TP HCM từ 2000 - 2021 
    Nhận được 1032 bộ dữ liệu ~ 3 năm

'''
data=requests.get('http://api.openweathermap.org/data/2.5/air_pollution/history?lat=10.7546664&lon=106.415029&start=946726894&end=1610129052&appid=9f7c977884b479967cee75682a860b6b')

In [ ]:
#Parse JSON
air_quality=json.loads(data.text)

In [47]:
#Test
air_quality['list'][0]

{'main': {'aqi': 4},
 'components': {'co': 988.01,
  'no': 18.33,
  'no2': 26.39,
  'o3': 0.4,
  'so2': 15.5,
  'pm2_5': 46.07,
  'pm10': 60.15,
  'nh3': 11.02},
 'dt': 1609977600}

In [50]:
#Simplize dicts
def make_flat_dict(p):
    res={};
    components=p['components']
    
    res['dt']=p['dt']
    
    
    res['co']=components['co']
    
    
    res['no']=components['no']
    
    
    res['no2']=components['no2']
    
    
    res['o3']=components['o3']
    
    
    res['so2']=components['so2']
    
    
    res['pm2_5']=components['pm2_5']
    
    
    res['pm10']=components['pm10']
    
  
    res['nh3']=components['nh3']
    
    
    res['aqi']=p['main']['aqi']
    
    return res
    
    pass

In [51]:
list_records=[]
for record in air_quality['list']:
    list_records.append(make_flat_dict(record))
    

In [53]:
#Write to csv
with open('air_quality.csv', 'w',newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=list_records[0].keys())
        writer.writeheader()
        for data in list_records:
            writer.writerow(data)

In [15]:
#Phần gsmarena chưa nỡ xóa

dict_keys(['main', 'components', 'dt'])

In [6]:
#Get brand link
'''
url = 'https://www.gsmarena.com/makers.php3'
html_text = requests.get(url).text

tree = BeautifulSoup(html_text, 'html.parser')
td=tree.select("td > a")

brand_href=['https://www.gsmarena.com/'+x['href'] for x in td]
brand_href
'''

In [28]:
#Get product link
'''
product_href=[]
for url in brand_href:
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text, 'html.parser')
    lis=tree.select("div.makers li > a")
    product_href.extend(['https://www.gsmarena.com/'+x['href'] for x in lis])
    print('new_one')
    print(url)
'''  

In [2]:
#Save product link
'''fw = open('product_link.csv', 'w')
for row in product_href:
    fw.write(f"{row}\n")
fw.close()'''
#Load from csv file
product_href=pd.read_csv('product_link.csv',sep='\n',header=None) 

In [35]:
#Get detail of a product
def get_detail(url):
    a_product={}
    product_list=[]
    
    html_text=requests.get(url).text
    tree=BeautifulSoup(html_text,'html.parser')
    
    try:
        a_product["release_day"]=tree.find("span", attrs={"data-spec": "released-hl"}).text.replace(',',';')
    except:
        a_product["release_day"]=''
    
    try:
        a_product["display_size"]=tree.find("span", attrs={"data-spec": "displaysize-hl"}).text[:-1].replace(',',';')
    except:
        pass
    
    try:
        a_product["cpu"]=tree.find("div", attrs={"data-spec": "chipset-hl"}).text.replace(',',';')
    except:
        a_product["cpu"]=''
        pass

    try:
        benchmark=tree.find("td", attrs={"data-spec": "tbench"}).text
        benchmark=benchmark.replace("\r\n",",")
        a_product["benchmark"]=benchmark[1:].replace(',',';')
    except:
        a_product["benchmark"]=''
        pass

    try:
        a_product["displayres"]=tree.find("div", attrs={"data-spec": "displayres-hl"}).text.replace(',',';')
    except:
        a_product["displayres"]=''
        pass

    try:
        a_product["camerapixels"]=tree.find("span", attrs={"data-spec": "camerapixels-hl"}).text.replace(',',';')
    except:
        a_product["camerapixels"]=''
        pass

    try:
        a_product["videopixels"]=tree.find("div", attrs={"data-spec": "videopixels-hl"}).text.replace(',',';')
    except:
        a_product["videopixels"]=''
        pass

    try:
        a_product["batsize"]=tree.find("span", attrs={"data-spec": "batsize-hl"}).text.replace(',',';')
    except:
        a_product["batsize"]=''
        pass

    try:
        price=tree.find("table", attrs={"class": "pricing inline widget"}).find_all("tr")
        for var in price:
            #ver={}
            a_product['spec']=var.find("td").text.replace(',',';')
            a_product['price']=var.find("a").text[2:].replace(',','')
            a_product['currency']=var.find("a").text[0:1].replace(',',';')
            #a_product["ver"]=ver
            product_list.append(a_product.copy())
    except:
        #ver={}
        a_product['spec']=''
        a_product['price']=''
        a_product['currency']=''
        #a_product["ver"]=ver
        product_list.append(a_product.copy())
        pass
    return product_list
    pass


In [38]:
dicta=get_detail(product_href[0][2991])

In [39]:
dicta

[{'release_day': 'Released 2020; November 27',
  'display_size': '6.53',
  'cpu': 'Snapdragon 662',
  'benchmark': 'AnTuTu: 177904 (v8);GeekBench: 1398 (v5.1);GFXBench: 5.9fps (ES 3.1 onscreen)',
  'displayres': '1080x2340 pixels',
  'camerapixels': '48',
  'videopixels': '1080p',
  'batsize': '6000',
  'spec': '64GB 4GB RAM',
  'price': '219.99',
  'currency': '$'},
 {'release_day': 'Released 2020; November 27',
  'display_size': '6.53',
  'cpu': 'Snapdragon 662',
  'benchmark': 'AnTuTu: 177904 (v8);GeekBench: 1398 (v5.1);GFXBench: 5.9fps (ES 3.1 onscreen)',
  'displayres': '1080x2340 pixels',
  'camerapixels': '48',
  'videopixels': '1080p',
  'batsize': '6000',
  'spec': '128GB 4GB RAM',
  'price': '239.99',
  'currency': '$'}]

In [25]:
product_href[0][2991]

'https://www.gsmarena.com/xiaomi_poco_m3-10599.php'

In [7]:
list(dicta[0].keys())

['release_day',
 'display_size',
 'cpu',
 'benchmark',
 'displayres',
 'camerapixels',
 'videopixels',
 'batsize',
 'spec',
 'price',
 'currency']

In [40]:
#Thử với 30 sản phẩm
product_list=[]
product_href_slice=product_href[0][3000:3005]
count=1
for url in product_href_slice:
    product_list.extend(get_detail(url))
    time.sleep(30)#set nhieu nhieu vao khong la bi too many request :(

In [43]:
(product_list)

[{'release_day': 'Released 2020; September 09',
  'display_size': '6.53',
  'cpu': 'Helio G25',
  'benchmark': '',
  'displayres': '720x1600 pixels',
  'camerapixels': '13',
  'videopixels': '1080p',
  'batsize': '5000',
  'spec': '32GB 2GB RAM',
  'price': '108.99',
  'currency': '€'},
 {'release_day': 'Released 2020; September 18',
  'display_size': '6.53',
  'cpu': 'Helio G25',
  'benchmark': '',
  'displayres': '720x1600 pixels',
  'camerapixels': '13',
  'videopixels': '1080p',
  'batsize': '5000',
  'spec': '64GB 4GB RAM',
  'price': '8893',
  'currency': '₹'},
 {'release_day': 'Released 2020; September 18',
  'display_size': '6.53',
  'cpu': 'Helio G25',
  'benchmark': '',
  'displayres': '720x1600 pixels',
  'camerapixels': '13',
  'videopixels': '1080p',
  'batsize': '5000',
  'spec': '128GB 4GB RAM',
  'price': '9770',
  'currency': '₹'},
 {'release_day': 'Released 2020; September 15',
  'display_size': '6.53',
  'cpu': 'Helio G80',
  'benchmark': '',
  'displayres': '1080x23

In [47]:
with open('specify_info_gasmarena.csv', 'w', newline='',encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(product_list[0].keys()))
    writer.writeheader()
    
    for i in range(5):
        product_href_slice=product_href[0][5*i:5*i+5]
        product_list=[]
        for url in product_href_slice:
            product_list.extend(get_detail(url))
            time.sleep(30)#set nhieu nhieu vao khong la bi too many request :(
        for data in product_list:
            print(data)
            writer.writerow(data)
        

{'release_day': 'Released 2018; July', 'display_size': '9.7', 'cpu': 'Rockchip RK3399', 'benchmark': '', 'displayres': '1536x2048 pixels', 'camerapixels': '5', 'videopixels': '720p', 'batsize': '4500', 'spec': '', 'price': '', 'currency': ''}
{'release_day': 'Released 2016; October', 'display_size': '7.0', 'cpu': 'MT8735', 'benchmark': '', 'displayres': '720x1280 pixels', 'camerapixels': '13', 'videopixels': '1080p', 'batsize': '3400', 'spec': '', 'price': '', 'currency': ''}
{'release_day': 'Released 2016; December', 'display_size': '5.5', 'cpu': 'MT6753', 'benchmark': '', 'displayres': '1080x1920 pixels', 'camerapixels': '13', 'videopixels': '1080p', 'batsize': '4080', 'spec': '', 'price': '', 'currency': ''}
{'release_day': 'Released 2016; December', 'display_size': '5.0', 'cpu': 'MT6737', 'benchmark': '', 'displayres': '720x1280 pixels', 'camerapixels': '8', 'videopixels': 'Video recorder', 'batsize': '2000', 'spec': '', 'price': '', 'currency': ''}
{'release_day': 'Released 2016; 

KeyboardInterrupt: 

In [ ]:
#Phan ong lam o ben duoi

In [3]:
browser = webdriver.Chrome(executable_path='./chromedriver.exe')

In [29]:
browser.get('https://www.gsmarena.com/makers.php3')

# Tắt cửa sổ quảng cáo nếu có bằng cách ấn phím Esc 
browser.find_element_by_css_selector('html').send_keys(Keys.ESCAPE)

In [30]:
array_brand = browser.find_element_by_css_selector('td > a')

In [31]:
array_brand = tree.select("td > a")

<selenium.webdriver.remote.webelement.WebElement (session="a903077cf2aab2f02cde4153e0e6a846", element="10c42893-e81d-4cfe-9389-1107b4973311")>

In [14]:
for i in array_brand:
    i.click()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=87.0.4280.88)


In [6]:
try:  
    for i in range(20):
        browser.find_element_by_class_name("viewmore").click()
except:
    pass

In [7]:
html_text = browser.page_source
browser.quit()

In [8]:
tree = str(BeautifulSoup(html_text, 'html.parser'))

dtdd_urls = re.findall(r'/dtdd/[\w\d-]+', tree)
dtdd_urls = dtdd_urls[0:150] # Loại bỏ những loại điện thoại phím bấm vì không có đủ các thông số như ram,rom,...
dtdd_urls = list(set(dtdd_urls)) # Xóa phần tử trùng lặp.
# print(dtdd_urls)
# print(len(dtdd_urls))

['/dtdd/realme-6i', '/dtdd/xiaomi-redmi-note-8', '/dtdd/nokia-6300-4g', '/dtdd/vivo-x50-pro', '/dtdd/samsung-galaxy-s20-fan-edition', '/dtdd/samsung-galaxy-a01', '/dtdd/realme-7-pro', '/dtdd/xiaomi-redmi-9', '/dtdd/realme-c11', '/dtdd/vsmart-star-4-16gb', '/dtdd/iphone-se-128gb-2020-hop-moi', '/dtdd/vivo-y50', '/dtdd/samsung-galaxy-a11', '/dtdd/oppo-a92', '/dtdd/xiaomi-redmi-note-8-128gb', '/dtdd/iphone-11-128gb', '/dtdd/nokia-24', '/dtdd/samsung-galaxy-z-fold-2', '/dtdd/nokia-c2', '/dtdd/vsmart-joy-4', '/dtdd/oppo-reno5', '/dtdd/xiaomi-redmi-note-9-pro', '/dtdd/vsmart-joy-4-4gb-64gb', '/dtdd/nokia-5310-2020', '/dtdd/iphone-se-64gb-2020-hop-moi', '/dtdd/samsung-galaxy-m51', '/dtdd/oppo-a15', '/dtdd/nokia-53', '/dtdd/vsmart-aris-pro', '/dtdd/samsung-galaxy-note-10-lite', '/dtdd/vsmart-active-3-6gb', '/dtdd/vsmart-star-4', '/dtdd/vsmart-bee-3', '/dtdd/oneplus-nord-n10-5g', '/dtdd/iphone-7-128gb', '/dtdd/iphone-12-pro-256gb', '/dtdd/vivo-y12s', '/dtdd/huawei-nova-7i', '/dtdd/iphone-12-min

In [9]:
fw = open('dtdd_file.csv', 'w')
fw.write(f"{'price'}\t{'screen'}\t{'ram'}\t{'rom'}\t{'pin'}\t{'camera'}\t{'star'}\n")

for i in range(len(dtdd_urls)):
    try:
        html_text = requests.get('https://thegioididong.com' + dtdd_urls[i]).text
        tree = BeautifulSoup(html_text, 'html.parser')

        text = tree.find('ul', {'class': 'parameter'}).text
        price = tree.find('div', {'class': 'area_price'}).text
        price = int(re.findall(r'([\d+.]+)', price)[0].replace('.',''))
        screen = float(re.findall(r'([\d+.]+)\"', text)[0])
        ram = int(re.findall(r'RAM:(\d+)', text)[0])
        rom = int(re.findall(r'Bộ nhớ trong:(\d+)', text)[0])
        pin = int(re.findall(r'pin:(\d+)', text)[0])
        camera = int(re.findall(r'sau:Chính (\d+)', text)[0])
        star = tree.find('div', {'class': 'lcrt'}).text
        star = float(re.findall(r'([\d+.]+)', star)[0])

        fw.write(f'{price}\t{screen}\t{ram}\t{rom}\t{pin}\t{camera}\t{star}\n')
    except:
        continue
fw.close()

In [10]:
df = pd.read_csv('dtdd_file.csv', sep='\t')
df

,price,screen,ram,rom,pin,camera,star
0,4690000,6.50,4,128,5000,48,3.3
1,17990000,6.56,8,256,4315,48,4.3
2,14990000,6.50,8,128,4500,12,1.5
3,2590000,5.70,2,16,3000,13,2.8
4,8990000,6.40,8,128,4500,64,4.5
...,...,...,...,...,...,...,...
90,3690000,6.52,4,64,5000,12,3.3
91,6990000,6.40,8,128,4025,48,3.5
92,2990000,6.50,4,64,5000,13,3.4
93,3590000,6.30,3,32,4000,48,3.0
